<a href="https://colab.research.google.com/github/giri14v/Alternative-Agriculture/blob/main/AI_StoryTeller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

project_dir = '/content/drive/MyDrive/AI_Storyteller'
if not os.path.exists(project_dir):
    os.makedirs(project_dir)

print(f"Project directory: {project_dir}")


In [6]:
# Install required libraries
!pip install torch torchvision
!pip install transformers Pillow
!pip install ftfy regex tqdm
!pip install torch torchvision
!pip install sentencepiece
!pip install streamlit
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 5.7 MB/s eta 0:00:00


In [7]:
%%writefile app.py
import streamlit as st
import torch
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration, AutoTokenizer, GPT2LMHeadModel

# Setup Streamlit app
st.title("Interactive AI Storyteller with Multiple Images (GPT-2)")

# Step 1: Upload multiple images from the user
uploaded_files = st.file_uploader("Upload multiple images", accept_multiple_files=True, type=['png', 'jpg', 'jpeg'])

# Step 2: Load BLIP model for image feature extraction
device = "cuda" if torch.cuda.is_available() else "cpu"
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

# Step 3: Load GPT-2 model for story generation
gpt2_tokenizer = AutoTokenizer.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)

# Placeholder to accumulate captions for all images
image_captions = []

# Check if images are uploaded
if uploaded_files:
    # Iterate over each uploaded image
    for idx, uploaded_file in enumerate(uploaded_files):
        # Load the image
        image = Image.open(uploaded_file)

        # Display the image to the user
        st.image(image, caption=f"Uploaded Image {idx + 1}", use_column_width=True)

        # Step 4: Extract features from the image using BLIP
        inputs = blip_processor(images=image, return_tensors="pt").to(device)
        with torch.no_grad():
            out = blip_model.generate(**inputs)
        image_caption = blip_processor.decode(out[0], skip_special_tokens=True)

        # Display the extracted caption
        st.write(f"Extracted features from Image {idx + 1}: {image_caption}")

        # Append the caption to the list of image captions
        image_captions.append(image_caption)

    # Debugging: Print extracted captions
    st.write(f"Extracted Captions: {image_captions}")

    # Step 5: Create a unified story prompt by combining all image captions with transitions
    combined_caption_prompt = " ".join(image_captions)

    # Update the prompt for better story guidance
    prompt = (
        f"Here are some scenes: {combined_caption_prompt}. "
        "Weave these elements together into a creative and imaginative story that connects the scenes, focusing on emotions, characters, and how these moments come together to form a beautiful narrative."
    )

    # Debugging: Print the prompt
    st.write(f"Story Generation Prompt: {prompt}")

    # Encode the prompt for GPT-2
    encoding = gpt2_tokenizer(prompt, return_tensors="pt").to(device)
    input_ids = encoding['input_ids']

    # Step 6: Generate the story from GPT-2
    gpt2_output = gpt2_model.generate(
        input_ids=input_ids,
        max_length=800,  # Increase max_length for a longer, more detailed story
        num_beams=5,     # Increase num_beams for higher quality output
        early_stopping=True,
        no_repeat_ngram_size=3,  # Avoid repeating phrases
        temperature=0.8  # Add some randomness for creativity
    )

    # Decode the generated story
    full_story = gpt2_tokenizer.decode(gpt2_output[0], skip_special_tokens=True)

    # Step 7: Display the final generated story
    st.subheader("Final Generated Story:")
    st.write(full_story)

# Instruction to the user if no images are uploaded
else:
    st.write("Please upload multiple images to start generating the interactive story.")


Overwriting app.py


In [2]:
!pip install pyngrok

In [3]:
!ngrok authtoken 2nYA63gQ3vZw3Pigi8ISB5b1fWR_5rWYehD8HsWVF8yJvV9dg


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!cat /root/.config/ngrok/ngrok.yml


cat: /root/.config/ngrok/ngrok.yml: No such file or directory


In [8]:
from pyngrok import ngrok

public_url = ngrok.connect(8501)
print(f"Access the Streamlit app here: {public_url}")


Access the Streamlit app here: NgrokTunnel: "https://fa25-34-138-149-161.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!streamlit run app.py &





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.138.149.161:8501

2024-10-19 06:27:50.231437: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-19 06:27:50.259697: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-19 06:27:50.268438: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-19 06:27:50.288177: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
T